In [1]:
import operator
from collections import deque

In [2]:
test_input = """Monkey 0:
  Starting items: 79, 98
  Operation: new = old * 19
  Test: divisible by 23
    If true: throw to monkey 2
    If false: throw to monkey 3

Monkey 1:
  Starting items: 54, 65, 75, 74
  Operation: new = old + 6
  Test: divisible by 19
    If true: throw to monkey 2
    If false: throw to monkey 0

Monkey 2:
  Starting items: 79, 60, 97
  Operation: new = old * old
  Test: divisible by 13
    If true: throw to monkey 1
    If false: throw to monkey 3

Monkey 3:
  Starting items: 74
  Operation: new = old + 3
  Test: divisible by 17
    If true: throw to monkey 0
    If false: throw to monkey 1"""

In [3]:
def parse_op(s):
    a, op, b = s.split("new = ")[-1].split(" ")

    op = operator.add if op == "+" else operator.mul
    self_operand = b == "old"

    return lambda x: op(x, x if self_operand else int(b))

In [10]:
def parse_monkeys(input):
    SEP = ": "

    monkeys = []

    for monkey_data in input.split("\n\n"):
        monkey_stuff = monkey_data.split('\n')

        # use iter and next here?
        items = deque([int(i) for i in monkey_stuff[1].split(SEP)[-1].split(", ")])
        operation = parse_op(monkey_stuff[2].split(SEP)[-1])
        test_div = int(monkey_stuff[3].split(" ")[-1])
        true_throw = int(monkey_stuff[4].split(" ")[-1])
        false_throw = int(monkey_stuff[5].split(" ")[-1])

        monkeys.append([items, operation, test_div, true_throw, false_throw])
    
    return monkeys

In [11]:
input = open("inputs/11").read()

In [12]:
def p1(s):
    monkeys = parse_monkeys(s)

    inspection_counts = [0] * len(monkeys)

    for round in range(20):
        for i, (items, operation, test_div, true_throw, false_throw) in enumerate(monkeys):
            while items:
                inspection_counts[i] += 1

                item = items.popleft()

                worry_level = operation(item) // 3

                throw_target = true_throw if worry_level % test_div == 0 else false_throw

                monkeys[throw_target][0].append(worry_level)
    
    srted = sorted(inspection_counts, reverse=True)
    return srted[0] * srted[1]

In [13]:
p1(test_input)

10605

In [17]:
def p2(s):
    monkeys = parse_monkeys(s)
    inspection_counts = [0] * len(monkeys)

    # ok, so we just replace the absolute worry levels with the worry levels mod the primes we're going to be testing by
    primes = [t[2] for t in monkeys]

    # each item is now going to be a dict of values mod p
    for m in monkeys:
        m[0] = deque([{p: i % p for p in primes} for i in m[0]])

    for round in range(10_000):
        for i, (items, operation, test_div, true_throw, false_throw) in enumerate(monkeys):
            while items:
                inspection_counts[i] += 1

                item = items.popleft()

                # apply the operation mod p
                for p, w in item.items():
                    item[p] = operation(w) % p

                throw_target = true_throw if item[test_div] == 0 else false_throw

                monkeys[throw_target][0].append(item)
    
    srted = sorted(inspection_counts, reverse=True)
    return srted[0] * srted[1]

In [19]:
# p2(test_input)
p2(input)

18085004878